In [1]:
import numpy as np
import pandas as pd

In [2]:
# retrieve the data
msolution_orders = pd.read_csv('MSolution Orders.csv')
nova_orders = pd.read_csv('nova orders.csv')
open_bars = pd.read_csv('order open bars.csv')

live_orders = pd.concat([msolution_orders, nova_orders])
open_bars

,Nova,datetime_ohlcv,MSolutions,datetime_ohlcv.1
0,"EURUSD,M2 (offline)",NaN,NaN,NaN
1,Date,2024.03.11,Date,2024.03.11
2,Time,12:22,Time,12:01
3,Open,1.0942,Open,NaN
4,High,1.09435,High,NaN
...,...,...,...,...
363,Open,1.0924,NaN,NaN
364,High,1.0925,NaN,NaN
365,Low,1.09224,NaN,NaN
366,Close,1.0925,NaN,NaN


In [65]:
nova_open_bars = open_bars.loc[:, ['Nova', 'datetime_ohlcv']]
msolutions_open_bars = open_bars.loc[:, ['MSolutions', 'datetime_ohlcv.1']]
msolutions_open_bars_no_dups = msolutions_open_bars.drop_duplicates()
# remove any rows in 'Nova' that contain the word "EURUSD"
open_bars_no_symbol_label = nova_open_bars[~nova_open_bars['Nova'].str.contains('EURUSD')]
open_bars_datetime_close = open_bars_no_symbol_label[open_bars_no_symbol_label['Nova'].str.contains('Date') 
                                                     | open_bars_no_symbol_label['Nova'].str.contains('Close') 
                                                     | open_bars_no_symbol_label['Nova'].str.contains('Time')]
open_bars_datetime_close.head(10)

,Nova,datetime_ohlcv
1,Date,2024.03.11
2,Time,12:22
6,Close,1.0941
9,Date,2024.03.11
10,Time,13:06
14,Close,1.0937
17,Date,2024.03.11
18,Time,14:04
22,Close,1.094
25,Date,2024.03.11


In [69]:
def massaging_df(df, name_col, datetime_col):
    # remove any rows in name_col that contain the word "EURUSD"
    open_bars_no_symbol_label = df[~df[name_col].str.contains('EURUSD')]
    open_bars_datetime_close = open_bars_no_symbol_label[open_bars_no_symbol_label[name_col].str.contains('Date') 
                                                        | open_bars_no_symbol_label[name_col].str.contains('Close') 
                                                        | open_bars_no_symbol_label[name_col].str.contains('Time')]
    # pivot the dataframe to be wide instead of long, so move all the unique values in the 'Nova' column to be their own columns
    pivot = open_bars_datetime_close.pivot(columns=name_col, values=datetime_col)
    # forward fill all of the columns
    pivot_filled = pivot.ffill()
    # drop the rows with nulls
    pivot_filled_na_dropped = pivot_filled.dropna()
    # drop duplicates
    pivot_filled_na_dropped_no_dups = pivot_filled_na_dropped.drop_duplicates()
    # concatenate 'Date' and 'Time' columns
    pivot_filled_na_dropped_no_dups['Date'] = pivot_filled_na_dropped_no_dups['Date'].astype(str)
    pivot_filled_na_dropped_no_dups['Time'] = pivot_filled_na_dropped_no_dups['Time'].astype(str)
    pivot_filled_na_dropped_no_dups['datetime'] = pivot_filled_na_dropped_no_dups['Date'] + ' ' + pivot_filled_na_dropped_no_dups['Time']
    pivot_filled_na_dropped_no_dups['datetime'] = pd.to_datetime(pivot_filled_na_dropped_no_dups['datetime'], format='%Y.%m.%d %H:%M')
    # group by datetime, and get the second value of the index for 'Close' column
    pivot_filled_na_dropped_no_dups['Close'] = pivot_filled_na_dropped_no_dups['Close'].astype(float)
    # group by datetime, and get the count
    pivot_count = pivot_filled_na_dropped_no_dups.groupby('datetime')['Close'].count()
    # join pivot_count with pivot_filled_na_dropped_no_dups
    pivot_filled_na_dropped_no_dups_joined = pivot_filled_na_dropped_no_dups.join(pivot_count, on='datetime', rsuffix='_count', how='left')
    not_a_dup = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Close_count'] < 2]
    with_dups = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Close_count'] > 1]
    pivot_filled_na_dropped_no_dups_grouped = with_dups.groupby('datetime').nth(1)
    appended_df = pd.concat([not_a_dup, pivot_filled_na_dropped_no_dups_grouped])
    appended_df_sorted = appended_df.sort_values(by='datetime')

    return appended_df_sorted



In [77]:
msolutions_open_bars_no_dups['MSolutions']

0         NaN
1        Date
2        Time
3        Open
4        High
        ...  
335    Volume
338      Time
340      High
346      Time
349       Low
Name: MSolutions, Length: 192, dtype: object

In [76]:
#remove nulls from the 'MSolutions' column
msolutions_open_bars_no_dups_na_dropped = msolutions_open_bars_no_dups.dropna()
print(msolutions_open_bars_no_dups.MSolutions.count(), msolutions_open_bars_no_dups_na_dropped.MSolutions.count())
# msolutions_open_bars_no_dups[~msolutions_open_bars_no_dups['MSolutions'].str.contains('EURUSD')]
msolutions_open_bars_no_dups[msolutions_open_bars_no_dups.isna()]

191 185


,MSolutions,datetime_ohlcv.1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
335,NaN,NaN
338,NaN,NaN
340,NaN,NaN
346,NaN,NaN


In [70]:
import warnings
warnings.filterwarnings('ignore')
# nova_df = massaging_df(nova_open_bars, 'Nova', 'datetime_ohlcv')
msolutions_df = massaging_df(msolutions_open_bars_no_dups, 'MSolutions', 'datetime_ohlcv.1')
msolutions_df

TypeError: bad operand type for unary ~: 'float'

In [61]:
nova_df

,Close,Date,Time,datetime,Close_count
6,1.0941,2024.03.11,12:22,2024-03-11 12:22:00,1
14,1.0937,2024.03.11,13:06,2024-03-11 13:06:00,2
22,1.0940,2024.03.11,14:04,2024-03-11 14:04:00,2
30,1.0943,2024.03.11,14:31,2024-03-11 14:31:00,2
38,1.0939,2024.03.11,15:15,2024-03-11 15:15:00,2
46,1.0935,2024.03.11,15:51,2024-03-11 15:51:00,2
54,1.0936,2024.03.11,15:52,2024-03-11 15:52:00,2
62,1.0926,2024.03.11,16:32,2024-03-11 16:32:00,2
70,1.0928,2024.03.11,16:35,2024-03-11 16:35:00,2
78,1.0926,2024.03.11,16:36,2024-03-11 16:36:00,2


In [ ]:
msolutions_df

In [ ]:
warnings.resetwarnings()

In [38]:
# pivot the dataframe to be wide instead of long, so move all the unique values in the 'Nova' column to be their own columns
nova_open_bars_pivot = nova_open_bars.pivot(columns='Nova', values='datetime_ohlcv')
# forward fill all of the columns
nova_open_bars_pivot_filled = nova_open_bars_pivot.ffill()
# drop the rows with nulls
nova_open_bars_pivot_filled_na_dropped = nova_open_bars_pivot_filled.dropna()
# drop duplicates
nova_open_bars_pivot_filled_na_dropped_no_dups = nova_open_bars_pivot_filled_na_dropped.drop_duplicates()
# concatenate 'Date' and 'Time' columns
nova_open_bars_pivot_filled_na_dropped_no_dups['Date'] = nova_open_bars_pivot_filled_na_dropped_no_dups['Date'].astype(str)
nova_open_bars_pivot_filled_na_dropped_no_dups['Time'] = nova_open_bars_pivot_filled_na_dropped_no_dups['Time'].astype(str)
nova_open_bars_pivot_filled_na_dropped_no_dups['datetime'] = nova_open_bars_pivot_filled_na_dropped_no_dups['Date'] + ' ' + nova_open_bars_pivot_filled_na_dropped_no_dups['Time']
nova_open_bars_pivot_filled_na_dropped_no_dups['datetime'] = pd.to_datetime(nova_open_bars_pivot_filled_na_dropped_no_dups['datetime'], format='%Y.%m.%d %H:%M')
# group by datetime, and get the second value of the index for 'Close' columne
nova_open_bars_pivot_filled_na_dropped_no_dups['Close'] = nova_open_bars_pivot_filled_na_dropped_no_dups['Close'].astype(float)
nova_open_bars_pivot_filled_na_dropped_no_dups_grouped = nova_open_bars_pivot_filled_na_dropped_no_dups.groupby('datetime').nth(1)
nova_open_bars_pivot_filled_na_dropped_no_dups_grouped.head(10)

Nova,Close,Date,Time,datetime
14,1.0937,2024.03.11,13:06,2024-03-11 13:06:00
22,1.0940,2024.03.11,14:04,2024-03-11 14:04:00
30,1.0943,2024.03.11,14:31,2024-03-11 14:31:00
38,1.0939,2024.03.11,15:15,2024-03-11 15:15:00
46,1.0935,2024.03.11,15:51,2024-03-11 15:51:00
54,1.0936,2024.03.11,15:52,2024-03-11 15:52:00
62,1.0926,2024.03.11,16:32,2024-03-11 16:32:00
70,1.0928,2024.03.11,16:35,2024-03-11 16:35:00
78,1.0926,2024.03.11,16:36,2024-03-11 16:36:00
86,1.0923,2024.03.11,16:48,2024-03-11 16:48:00


In [53]:
pivot_count = nova_open_bars_pivot_filled_na_dropped_no_dups.groupby('datetime')['Close'].count()
# join pivot_count with pivot_filled_na_dropped_no_dups
pivot_filled_na_dropped_no_dups_joined = nova_open_bars_pivot_filled_na_dropped_no_dups.join(pivot_count, on='datetime', rsuffix='_count', how='left')
not_a_dup = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Close_count'] < 2]
with_dups = pivot_filled_na_dropped_no_dups_joined[pivot_filled_na_dropped_no_dups_joined['Close_count'] > 1]
pivot_filled_na_dropped_no_dups_grouped = with_dups.groupby('datetime').nth(1)

In [57]:
not_a_dup

,Close,Date,Time,datetime,Close_count
6,1.0941,2024.03.11,12:22,2024-03-11 12:22:00,1
114,1.0921,2024.03.11,18:08,2024-03-11 18:08:00,1
130,1.0917,2024.03.11,18:44,2024-03-11 18:44:00,1
186,1.0929,2024.03.11,20:08,2024-03-11 20:08:00,1
209,1.0923,2024.03.12,21:46,2024-03-12 21:46:00,1
250,1.0936,2024.03.12,6:40,2024-03-12 06:40:00,1
306,1.0936,2024.03.12,9:55,2024-03-12 09:55:00,1


In [56]:
pivot_filled_na_dropped_no_dups_grouped

,Close,Date,Time,datetime,Close_count
14,1.0937,2024.03.11,13:06,2024-03-11 13:06:00,2
22,1.0940,2024.03.11,14:04,2024-03-11 14:04:00,2
30,1.0943,2024.03.11,14:31,2024-03-11 14:31:00,2
38,1.0939,2024.03.11,15:15,2024-03-11 15:15:00,2
46,1.0935,2024.03.11,15:51,2024-03-11 15:51:00,2
54,1.0936,2024.03.11,15:52,2024-03-11 15:52:00,2
62,1.0926,2024.03.11,16:32,2024-03-11 16:32:00,2
70,1.0928,2024.03.11,16:35,2024-03-11 16:35:00,2
78,1.0926,2024.03.11,16:36,2024-03-11 16:36:00,2
86,1.0923,2024.03.11,16:48,2024-03-11 16:48:00,2


In [55]:
pivot_filled_na_dropped_no_dups_joined

,Close,Date,Time,datetime,Close_count
6,1.0941,2024.03.11,12:22,2024-03-11 12:22:00,1
10,1.0941,2024.03.11,13:06,2024-03-11 13:06:00,2
14,1.0937,2024.03.11,13:06,2024-03-11 13:06:00,2
18,1.0937,2024.03.11,14:04,2024-03-11 14:04:00,2
22,1.0940,2024.03.11,14:04,2024-03-11 14:04:00,2
...,...,...,...,...,...
350,1.0928,2024.03.12,11:20,2024-03-12 11:20:00,2
354,1.0928,2024.03.12,11:21,2024-03-12 11:21:00,2
358,1.0927,2024.03.12,11:21,2024-03-12 11:21:00,2
362,1.0927,2024.03.12,11:53,2024-03-12 11:53:00,2


In [39]:
# remove the last 16 rows of the 'MSolutions' dataframe


,MSolutions,datetime_ohlcv.1
1,Date,2024.03.11
2,Time,12:01
6,Close,NaN
10,Time,12:08
14,Close,1.0942
...,...,...
322,Time,10:17
330,Time,10:30
338,Time,10:42
346,Time,11:53


In [25]:

# pivot the dataframe to be wide instead of long, so move all the unique values in the 'msolutions' column to be their own columns
msolutions_open_bars_pivot = msolutions_open_bars_no_na.pivot(columns='MSolutions', values='datetime_ohlcv.1')
# forward fill all of the columns
msolutions_open_bars_pivot_filled = msolutions_open_bars_pivot.ffill()
# drop the rows with nulls
msolutions_open_bars_pivot_filled_na_dropped = msolutions_open_bars_pivot_filled.dropna()
# drop duplicates
msolutions_open_bars_pivot_filled_na_dropped_no_dups = msolutions_open_bars_pivot_filled_na_dropped.drop_duplicates()
msolutions_open_bars_pivot_filled_na_dropped_no_dups.head(10)

MSolutions,Close,Date,High,Low,Open,Time,Volume
15,1.0942,2024.03.11,1.09441,1.0942,1.0943,12:08,70
18,1.0942,2024.03.11,1.09441,1.0942,1.0943,13:11,70
19,1.0942,2024.03.11,1.09441,1.0942,1.0937,13:11,70
20,1.0942,2024.03.11,1.0938,1.0942,1.0937,13:11,70
21,1.0942,2024.03.11,1.0938,1.09363,1.0937,13:11,70
22,1.0938,2024.03.11,1.0938,1.09363,1.0937,13:11,70
23,1.0938,2024.03.11,1.0938,1.09363,1.0937,13:11,113
26,1.0938,2024.03.11,1.0938,1.09363,1.0937,14:00,113
27,1.0938,2024.03.11,1.0938,1.09363,1.0941,14:00,113
28,1.0938,2024.03.11,1.0942,1.09363,1.0941,14:00,113
